# 🤖 BERT 금융 뉴스 감성 분석 실습 문제지

**스토리:**  
핀테크 스타트업 ‘FinLens’는 실시간 금융 뉴스 감성 분석 서비스를 만들고자 합니다.  
당신은 이 서비스의 **AI 인턴**으로 채용되어, 다음을 구현해야 합니다.

1. 한국어 금융 뉴스를 **긍정·부정·중립**으로 분류하는 BERT 모델 파인튜닝  
2. 헤드라인 간 **NLI(함의·모순·중립)** 판별  
3. 공시 문서에서 “기준금리”를 추출하는 **MRC(QA)** 위젯  

각 셀에 제공된 부분 코드를 완성하고, **TODO** 표시된 부분을 채워주세요.

## 1. 환경 설정
필요한 라이브러리를 설치하는 셀입니다.

In [4]:
# 설치할 라이브러리 목록
!pip install -q \
    torch>=2.2 \
    transformers>=4.40 \
    datasets>=2.19 \
    scikit-learn \
    pandas \
    tqdm \
    evaluate

In [5]:
# 디바이스 설정 (CUDA → MPS → CPU)
import torch
device = torch.device("cuda" if torch.cuda.is_available() else
                      "mps" if torch.backends.mps.is_available() else
                      "cpu")
print("Using device:", device)

Using device: cuda


In [6]:
!pip install numpy<2.0


/bin/bash: line 1: 2.0: No such file or directory


## 2. 데이터 다운로드 및 전처리

In [7]:
import os
import pandas as pd

CSV_URL = "https://raw.githubusercontent.com/ukairia777/finance_sentiment_corpus/main/finance_data.csv"
CSV_PATH = "finance_data.csv"

# 데이터 다운로드
if not os.path.exists(CSV_PATH):
    df_remote = pd.read_csv(CSV_URL)
    df_remote.to_csv(CSV_PATH, index=False)
    print("다운로드 완료")

# 데이터 로드
df = pd.read_csv(CSV_PATH)
print("원본 컬럼:", df.columns.tolist())

# 컬럼명 변경
df = df.rename(columns={"labels": "label"})
print("수정 후 컬럼:", df.columns.tolist())

# 데이터 분포 확인
print("샘플 수:", len(df))
print(df["label"].value_counts())

원본 컬럼: ['labels', 'sentence', 'kor_sentence']
수정 후 컬럼: ['label', 'sentence', 'kor_sentence']
샘플 수: 4846
label
neutral     2879
positive    1363
negative     604
Name: count, dtype: int64


## 3. Datasets 로딩 및 학습/검증 분할

In [8]:
from datasets import Dataset, DatasetDict
from sklearn.model_selection import train_test_split

# pandas → HF Dataset
ds_all = Dataset.from_pandas(df, preserve_index=False)

# stratified split
train_idx, valid_idx = train_test_split(
    list(range(len(ds_all))),
    test_size=0.1,
    random_state=42,
    stratify=ds_all["label"]
)

ds = DatasetDict({
    "train": ds_all.select(train_idx),
    "validation": ds_all.select(valid_idx)
})

# 문자열 라벨 → 정수 인코딩
ds = ds.class_encode_column("label")
label_feature = ds["train"].features["label"]
label2id = {name: idx for idx, name in enumerate(label_feature.names)}
id2label = {idx: name for name, idx in label2id.items()}

print("label2id:", label2id)
print("id2label:", id2label)

Flattening the indices:   0%|          | 0/4361 [00:00<?, ? examples/s]

Casting to class labels:   0%|          | 0/4361 [00:00<?, ? examples/s]

Flattening the indices:   0%|          | 0/485 [00:00<?, ? examples/s]

Casting to class labels:   0%|          | 0/485 [00:00<?, ? examples/s]

label2id: {'negative': 0, 'neutral': 1, 'positive': 2}
id2label: {0: 'negative', 1: 'neutral', 2: 'positive'}


## 4. 토크나이징

In [9]:
from transformers import AutoTokenizer

MODEL_NAME = "klue/bert-base"
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)

def tokenize_batch(batch):
    # 'kor_sentence' 컬럼을 사용해 토크나이징
    return tokenizer(batch["kor_sentence"], truncation=True, padding="max_length", max_length=128)

# 토크나이징 적용
ds_tok = ds.map(tokenize_batch, batched=True)

# 텍스트 컬럼 제거, 포맷 설정
ds_tok = ds_tok.remove_columns(["sentence", "kor_sentence"])
ds_tok.set_format(type="torch", columns=["input_ids", "attention_mask", "label"])

print(ds_tok)

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Map:   0%|          | 0/4361 [00:00<?, ? examples/s]

Map:   0%|          | 0/485 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['label', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 4361
    })
    validation: Dataset({
        features: ['label', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 485
    })
})


## 5. 모델 준비 및 평가 함수

In [10]:
import torch
import numpy as np
import evaluate
from transformers import AutoModelForSequenceClassification, TrainingArguments, Trainer, set_seed

set_seed(42)
model = AutoModelForSequenceClassification.from_pretrained(
    MODEL_NAME,
    num_labels=len(label2id),
    id2label=id2label,
    label2id=label2id
)

accuracy = evaluate.load("accuracy")
def compute_metrics(eval_pred):
    logits, labels = eval_pred
    preds = np.argmax(logits, axis=-1)
    return accuracy.compute(predictions=preds, references=labels)

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at klue/bert-base and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


## 6. 학습

In [11]:
!pip install --upgrade datasets


In [12]:
from transformers import AutoModelForSequenceClassification, TrainingArguments, Trainer
import evaluate
import numpy as np

# 평가 지표
accuracy = evaluate.load("accuracy")

def compute_metrics(eval_pred):
    logits, labels = eval_pred
    preds = np.argmax(logits, axis=-1)
    return accuracy.compute(predictions=preds, references=labels)

# 모델 로드
model = AutoModelForSequenceClassification.from_pretrained(
    MODEL_NAME,
    num_labels=len(label2id),
    id2label=id2label,
    label2id=label2id
)

training_args = TrainingArguments(
    output_dir="bert_finance",
    num_train_epochs=3,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=32,
    learning_rate=2e-5,
    eval_strategy="epoch",
    logging_strategy="epoch",
    save_strategy="epoch",
    load_best_model_at_end=True,
    metric_for_best_model="accuracy",
    seed=42
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=ds_tok["train"],
    eval_dataset=ds_tok["validation"],
    tokenizer=tokenizer,
    compute_metrics=compute_metrics
)

train_result = trainer.train()
print("학습 결과:", train_result.metrics)

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at klue/bert-base and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
<ipython-input-12-f34f8cc12804>:35: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(
wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.
wandb: Currently logged in as: anony-moose-510502116165568808 to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


Epoch,Training Loss,Validation Loss,Accuracy
1,0.500800,0.423519,0.820619
2,0.257800,0.442012,0.843299
3,0.151500,0.486059,0.837113


학습 결과: {'train_runtime': 332.0383, 'train_samples_per_second': 39.402, 'train_steps_per_second': 2.467, 'total_flos': 860578211033856.0, 'train_loss': 0.30335162003252825, 'epoch': 3.0}


eval/accuracy,▁█▆
eval/loss,▁▃█
eval/runtime,█▇▁
eval/samples_per_second,▁▂█
eval/steps_per_second,▁▂█
train/epoch,▁▁▅▅███
train/global_step,▁▁▅▅███
train/grad_norm,▁▄█
train/learning_rate,█▄▁
train/loss,█▃▁
eval/accuracy,0.83711


Epoch,Training Loss,Validation Loss,Accuracy
1,0.189300,0.619690,0.820619
2,0.099900,0.657714,0.851546
3,0.038300,0.790098,0.835052


{'train_runtime': 352.4466, 'train_samples_per_second': 37.121, 'train_steps_per_second': 2.324, 'total_flos': 860578211033856.0, 'train_loss': 0.10916214082412813, 'epoch': 3.0}


## 7. 예측 함수 및 데모

In [13]:
def predict_finance_sentiment(text: str) -> dict:
    model.eval()
    inputs = tokenizer(text, return_tensors="pt", truncation=True, max_length=128)
    inputs = {k: v.to(device) for k, v in inputs.items()}
    with torch.no_grad():
        logits = model(**inputs).logits
    probs = torch.softmax(logits, dim=-1).squeeze().tolist()
    pred_id = int(np.argmax(probs))
    return {"label": id2label[pred_id], "confidence": round(probs[pred_id], 4)}

for s in [
    "이 회의 결과는 매우 긍정적이다.",
    "규제 리스크로 주가가 급락했다.",
    "금통위는 금리를 동결했다."
]:
    print(s, "→", predict_finance_sentiment(s))

이 회의 결과는 매우 긍정적이다. → {'label': 'positive', 'confidence': 0.9977}
규제 리스크로 주가가 급락했다. → {'label': 'negative', 'confidence': 0.9987}
금통위는 금리를 동결했다. → {'label': 'neutral', 'confidence': 0.996}


## 8. NLI 시연하기



In [14]:
from transformers import pipeline

# NLI 파이프라인
nli_pipe = pipeline("text-classification", model="Huffon/klue-roberta-base-nli", tokenizer="klue/roberta-base")
print(nli_pipe("삼성전자의 실적이 좋았다.</s></s>삼성전자는 부진했다."))

from transformers import pipeline

# 질문-응답(Machine Reading Comprehension) 파이프라인
qa_pipe = pipeline("question-answering", model="klue/roberta-base", tokenizer="klue/roberta-base")

# 예시 context와 question
context = "삼성전자는 2024년 1분기 매출 80조 원을 기록하며 전년 대비 실적이 대폭 상승했다."
question = "삼성전자의 1분기 매출은 얼마인가요?"

# 실행
print(qa_pipe(question=question, context=context))


Device set to use cuda:0


[{'label': 'CONTRADICTION', 'score': 0.9992614388465881}]


Some weights of RobertaForQuestionAnswering were not initialized from the model checkpoint at klue/roberta-base and are newly initialized: ['qa_outputs.bias', 'qa_outputs.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Device set to use cuda:0


{'score': 0.003082706592977047, 'start': 0, 'end': 22, 'answer': '삼성전자는 2024년 1분기 매출 80조'}


## 9. MRC 시연하기


In [16]:
from transformers import pipeline

# MRC 파이프라인 정의
qa_pipe = pipeline("question-answering", model="ainize/klue-bert-base-mrc", tokenizer="klue/bert-base")

# 문맥과 질문
context = (
    "한국은행 금융통화위원회는 2025년 5월 기준금리를 3.5%로 동결하였다. "
    "위원들은 세계 경제 둔화와 소비 부진을 이유로 관망세를 유지하기로 했다."
)
question = "2025년 5월 기준금리는 얼마인가?"

# 질문-응답 실행
result = qa_pipe(question=question, context=context)
print(result)


Device set to use cuda:0


{'score': 0.981529712677002, 'start': 29, 'end': 33, 'answer': '3.5%'}


# NLI(Natural Language Inference, 자연어 추론)는
두 문장 간의 의미 관계를 판단하는 대표적인 자연어 처리 과제입니다.

정의
주어진 문장(전제, Premise)과 다른 문장(가설, Hypothesis) 쌍이

Entailment(함의): 전제가 가설을 논리적으로 지지할 때

Contradiction(모순): 전제가 가설과 상반될 때

Neutral(중립): 전제만으로 가설의 진위 여부를 확정할 수 없을 때
위 세 가지 범주 중 하나로 분류하는 작업을 말합니다


# MRC는
“Machine Reading Comprehension(기계 독해)”의 약자로, 기계가 주어진 지문(문서)을 읽고 그 내용에 대한 질문에 답하는 능력을 평가·연습하는 자연어 처리 과제입니다. 주요 내용은 다음과 같습니다.

## 정의 및 목적

기계 독해 과제: 시스템에 지문(passage)과 질문(question)을 입력하면, 정답(answer)을 출력하도록 학습시킴.

목표: 문맥을 이해하고, 핵심 정보를 찾아내며, 자연어 질문에 정확히 응답할 수 있는 능력을 개발.

응용: 검색엔진, 챗봇, QA 시스템, 문서 요약 등 웹 검색을 수행하지 않았습니다

## 과제 유형

Extractive QA (추출식): 정답이 지문 내 특정 구간(span)으로 존재 → SQuAD, KorQuAD 등

Abstractive QA (생성식): 정답을 지문 내용을 바탕으로 생성 → NarrativeQA, TriviaQA

Multiple-choice QA: 선택지 중 정답 선택 → RACE, MCTest

Cloze-style QA: 빈칸 완성 → CBT(Cloze by Text), CLOTH

## 대표 데이터셋

SQuAD (Stanford Question Answering Dataset): 영어 추출식 QA 벤치마크

KorQuAD: 한국어 추출식 QA 데이터셋